<a href="https://colab.research.google.com/github/bajpaiarjun0333/CRNN-Keras/blob/master/NormalizedeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import numpy as np 
import pandas as pd

In [80]:
#dataset loading 
from keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train=x_train.reshape(x_train.shape[0],(x_train.shape[1]*x_train.shape[2]))
x_test=x_test.reshape(x_test.shape[0],(x_test.shape[1]*x_test.shape[2]))
x_train=x_train/255
x_test=x_test/255


In [81]:
def sigmoid(z):
  for i in range(z.shape[0]):
    idx=np.argmax(z[i])
  return 1/(1+np.exp(-z))

In [82]:
def sigmoid_derivative(z):
  return np.multiply((1/(1+np.exp(-z))),(1-(1/(1+np.exp(-z)))))

In [83]:
def softmax(x):
  for i in range(x.shape[0]):
    sum=0
    for j in range(x.shape[1]):
      sum=sum+np.exp(x[i][j])
    x[i]=np.exp(x[i])/sum
  return x

In [84]:
def cross_entropy(yhat,y_train):
  sum=0 
  for i in range(y_train.shape[0]):
    sum+=-((np.log2(yhat[i][y_train[i]])))
  return sum

In [85]:
def make_layer(w,b,input,output):
  weights=np.random.uniform(-1,1,(input,output))
  bias=np.random.uniform(-1,1,(1,output))
  w.append(weights)
  b.append(bias)
  return (w,b)

In [86]:
def forward_pass(x,w,b):
  a=[]
  h=[]
  l=len(w)
  #use a for loop for the required functionality
  for i in range (l-1):
    a1=np.add(np.matmul(x,w[i]),b[i])
    h1=sigmoid(a1)
    a.append(a1)
    h.append(h1)
    x=h1
  #this loop will complete the forward pass
  #for the last layer
  a1=np.add(np.matmul(x,w[l-1]),b[l-1])
  h1=softmax(a1)
  a.append(a1)
  h.append(h1)
  return (a,h)

In [87]:
def backward_pass(w,b,a,h,yhat,y_train,x_train,no_of_classes):
  #yahan par derivative ulte calculate honge yeh yaad rakhna,to saare loop ulte chalane honge humko
  #apne ko yl banana padega for all the n inputs
  #last layer ke liye special treatment
  wd=[]
  bd=[]
  ad=[]
  hd=[]
  el=np.zeros((y_train.shape[0],no_of_classes))
  for i in range (y_train.shape[0]):
      el[i][y_train[i]]=1

  yhatl=np.zeros((yhat.shape[0],1))
  for i in range (yhat.shape[0]):
    yhatl[i]=yhat[i][y_train[i]]
  #calculate for the last layer
  hd1=-(el/yhatl)
  ad1=-(el-yhat)
  hd.append(hd1)
  ad.append(ad1)
  #ab loop me daal kar saare ke saare humesha last appended wala hi use hoga
  l=len(w)
  for i in range(l-1,-1,-1):
    # print("i "+str(i))
    # print(h[i-1].shape)
    #print(ad[len(ad)-1].shape)
    q=h[i-1].T
    if i==0:
      q=x_train.T
    wi=np.matmul(q,ad[len(ad)-1])/x_train.shape[0]
    bi=np.sum(ad[len(ad)-1],axis=0)/x_train.shape[0]
    if i!=0:
      hd1=np.matmul(ad[len(ad)-1],w[i].T)
      sig=sigmoid_derivative(a[i-1])
      ad1=np.multiply(hd1,sig)
      hd.append(hd1)
      ad.append(ad1)
    wd.append(wi)
    bd.append(bi)
  return (wd,bd)  

In [88]:
def test(x_test,y_test,w,b):
  a,h=forward_pass(x_test,w,b)
  l=len(w)
  ypred=np.argmax(h[l-1],axis=1)
  count=0
  for i in range(y_test.shape[0]):
    if ypred[i]!=y_test[i]:
      count=count+1
  print((x_test.shape[0]-count)/y_test.shape[0])


In [89]:
def gradientDescent(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n):
  for i in range (iter):
    a=[]
    h=[]
    a,h=forward_pass(x_train,w,b,a,h)
    #print(a[0][:])
    #print(a[l-1][:])
    loss=cross_entropy(h[l-1],y_train)
    print("Iteration Number: "+str(i)+" loss: "+str(loss/x_train.shape[0]))
    wd,bd=backward_pass(w,b,a,h,h[l-1],y_train,x_train)
    for j in range (l):
      w[j]=w[j]-n*wd[l-1-j]
      b[j]=b[j]-n*bd[l-1-j]
  

  print("Test Results: ")
  test(x_test,y_test,w,b)




In [90]:
def createBatches(x_train,y_train,batchSize):
  data=[]
  ans=[]
  for i in range(int(x_train.shape[0]/batchSize)):
    batch=[]
    batch_ans=[]
    for j in range(i*batchSize,min((i+1)*batchSize,x_train.shape[0]),1):
      batch.append(x_train[j])
      batch_ans.append(y_train[j])
    batch=np.array(batch)
    batch_ans=np.array(batch_ans)
    data.append(batch)
    ans.append(batch_ans)
  return data,ans


In [91]:
def MGradientDescent(x_train,y_train,no_of_classes,w,b,l,n):
  a,h=forward_pass(x_train,w,b)
  wd,bd=backward_pass(w,b,a,h,h[l-1],y_train,x_train,no_of_classes)
  return (wd,bd)

In [92]:
def momentumGradientDescent(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n,batchSize,beta):
  data,ans=createBatches(x_train,y_train,batchSize)
  #batches have been made accordingly
  moment=[]
  momentB=[]
  for i in range(l):
    temp=np.zeros((w[i].shape))
    temp2=np.zeros(b[i].shape)
    moment.append(temp)
    momentB.append(temp2)
  print(len(data))
  for i in range(iter):
    print("Iteration No: "+str(i))
    for j in range(len(data)):

      # if j%10==0:
      #   print("Batch Number "+str(j))
      wd,bd=MGradientDescent(data[j],ans[j],no_of_classes,w,b,l,n)
      for k in range (l):
        moment[k]=(moment[k]*beta)+wd[l-1-k]
        momentB[k]=(momentB[k]*beta)+bd[l-1-k]
        w[k]=w[k]-n*moment[k]
        b[k]=b[k]-n*momentB[k]
  
    a,h=forward_pass(x_train,w,b)
    loss=cross_entropy(h[l-1],y_train)
    print("Iteration Number: "+str(i)+" loss: "+str(loss/x_train.shape[0]))
    print("Test Results: ")
    test(x_test,y_test,w,b) 




In [93]:
def batchGrad(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n,batchSize):
  data,ans=createBatches(x_train,y_train,batchSize)
  #start the batch gradient descent 
  for i in range(iter):
    h=None
    for j in range(len(data)):
      wd,bd=MGradientDescent(data[j],ans[j],no_of_classes,w,b,l,n)
    a=[]
    h=[]
    a,h=forward_pass(x_train,w,b)
    loss=cross_entropy(h[l-1],y_train)
    print("Iteration Number: "+str(i)+" loss: "+str(loss/x_train.shape[0]))
    print("Test Results: ")
    test(x_test,y_test,w,b) 



In [116]:
def nsGradientDescent(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n,batchSize,beta):
  #batches have been made accordingly
  data,ans=createBatches(x_train,y_train,batchSize)
  moment=[]
  momentB=[]
  for i in range(l):
    temp=np.zeros((w[i].shape))
    temp2=np.zeros((b[i].shape))
    moment.append(temp)
    momentB.append(temp2)
  for i in range(iter):
    print("Iteration No: "+str(i))
    for j in range(len(data)):
      for k in range(l):
        w[k]=w[k]-beta*moment[k]
        b[k]=b[k]-beta*momentB[k]
      wd,bd=MGradientDescent(data[k],ans[k],no_of_classes,w,b,l,n)
      for k in range (l):
        moment[k]=beta*moment[k]+n*wd[l-1-k]
        momentB[k]=beta*momentB[k]+n*bd[l-1-k]
        w[k]=w[k]-moment[k]
        b[k]=b[k]-momentB[k]
      
    a,h=forward_pass(x_train,w,b)
    loss=cross_entropy(h[l-1],y_train)
    print("Iteration Number: "+str(i)+" loss: "+str(loss/x_train.shape[0]))
    print("Test Results: Nestorov Gradient Descent :: ")
    test(x_test,y_test,w,b) 

In [121]:
def architecture(x_train,y_train,x_test,y_test,no_of_classes):

  w=[]
  b=[]
  w,b=make_layer(w,b,784,128)
  w,b=make_layer(w,b,128,128)
  w,b=make_layer(w,b,128,128)
  w,b=make_layer(w,b,128,10)
  l=len(w)
  iter=5
  n=0.1
  #gradientDescent(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n)
  #print("Batch Gradient Descent: ")
  #batchGrad(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n,1024)
  #print("Momentum Gradient Descent: ")
  #momentumGradientDescent(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n,1024,0.9)
  #print("Starting of Nestrov Gradient Descent: ")
  nsGradientDescent(x_train,y_train,x_test,y_test,no_of_classes,w,b,l,iter,n,4096,0.9)
  

In [122]:
architecture(x_train,y_train,x_test,y_test,10)

Iteration No: 0
Iteration Number: 0 loss: 2.7452287105748177
Test Results: Nestorov Gradient Descent :: 
0.5272
Iteration No: 1
Iteration Number: 1 loss: 1.3240190563172218
Test Results: Nestorov Gradient Descent :: 
0.6512
Iteration No: 2
Iteration Number: 2 loss: 1.002290291262035
Test Results: Nestorov Gradient Descent :: 
0.7316
Iteration No: 3
Iteration Number: 3 loss: 0.9079109453828428
Test Results: Nestorov Gradient Descent :: 
0.7558
Iteration No: 4
Iteration Number: 4 loss: 0.8602221207162184
Test Results: Nestorov Gradient Descent :: 
0.7686
